In [8]:
!pip install praw
!pip install psaw

In [9]:
!pip install tqdm

In [10]:
import praw
import pandas as pd
from psaw import PushshiftAPI
from datetime import datetime
import time
from tqdm import tqdm

In [15]:
# Reddit API credentials (replace with your own)
REDDIT_CLIENT_ID = "hMGPCziTpNJ_f_2e0kqnlA"
REDDIT_CLIENT_SECRET = "2JEZrTqpkziQAygS4r9mGLo0Uc3iag"
REDDIT_USER_AGENT = "windows:BC_EOL_Scrape:v1.0 (by /u/Organic_Ear_9037)"

In [27]:
#Death and Dying Flair
reddit = praw.Reddit(
    client_id=REDDIT_CLIENT_ID,
    client_secret=REDDIT_CLIENT_SECRET,
    user_agent=REDDIT_USER_AGENT
)

subreddit_name = "cancer"
flair_text = "Death and Dying"
post_limit = 1000  # Max number of posts to fetch

# Store extracted posts
posts = []

print(f"Fetching up to {post_limit} posts with flair '{flair_text}'...")

# Initialize progress bar
with tqdm(total=post_limit, desc="Processing posts", unit="post") as pbar:
    for submission in reddit.subreddit(subreddit_name).new(limit=post_limit):
        pbar.update(1)

        # Check if the post has the correct flair
        if submission.link_flair_text and submission.link_flair_text.lower() == flair_text.lower():
            posts.append({
                "Title": submission.title,
                "Text": submission.selftext,
                "Date": datetime.utcfromtimestamp(submission.created_utc).strftime('%Y-%m-%d %H:%M:%S'),
                "URL": submission.url,
                "Upvotes": submission.score,
                "Comments": submission.num_comments
            })

# Convert to DataFrame
df = pd.DataFrame(posts)

# Save to CSV
df.to_csv("breastcancer_death_dying_flair_posts.csv", index=False)

print("Data extraction complete.")

Fetching up to 1000 posts with flair 'Death and Dying'...


Processing posts:  93%|██████████████████▋ | 932/1000 [00:12<00:00, 75.01post/s]

Data extraction complete.


In [18]:
df

""


In [28]:
# Initialize PRAW Reddit instance
reddit = praw.Reddit(
    client_id=REDDIT_CLIENT_ID,
    client_secret=REDDIT_CLIENT_SECRET,
    user_agent=REDDIT_USER_AGENT
)

# Define subreddit and keywords
subreddit_name = "cancer"
keywords = ["death", "dying", "palliative", "hospice", "terminal", "metastatic", "metastasized", "metastasis", "stage IV", "stage 4", "eol", "end of life", "comfort care"]

# Store extracted posts
posts = []
post_limit = 1000  # Maximum posts to attempt to fetch
batch_size = 100  # Fetch 100 at a time (pagination limit)
processed = 0  # Track processed posts

print(f"Fetching up to {post_limit} posts...")

# Initialize progress bar
with tqdm(total=post_limit, desc="Processing posts", unit="post") as pbar:
    after = None  # Track pagination
    
    while processed < post_limit:
        try:
            # Fetch next batch of posts
            submissions = reddit.subreddit(subreddit_name).new(limit=post_limit)
            if not submissions:  # Stop if no more posts are available
                print("No more posts to retrieve.")
                break

            for submission in submissions:
                pbar.update(1)  # Update progress bar
                processed += 1

                # Check if post contains keywords
                if any(keyword in submission.title.lower() or keyword in submission.selftext.lower() for keyword in keywords):
                    posts.append({
                        "Title": submission.title,
                        "Text": submission.selftext,
                        "Date": datetime.utcfromtimestamp(submission.created_utc).strftime('%Y-%m-%d %H:%M:%S'),
                        "URL": submission.url
                    })

            # Update "after" parameter for pagination
            after = submissions[-1].fullname  # Get ID of the last post in batch

            # Save progress every 500 posts
            if processed % 500 == 0:
                df = pd.DataFrame(posts)
                df.to_csv("breastcancer_death_posts.csv", index=False)

        except Exception as e:
            print(f"Error: {e}. Waiting and retrying...")
            time.sleep(5)  # Wait before retrying

# Final save to CSV
df = pd.DataFrame(posts)
df.to_csv("breastcancer_death_posts.csv", index=False)

print("Data extraction complete.")

Fetching up to 1000 posts...


Processing posts:  90%|██████████████████  | 901/1000 [00:13<00:01, 94.58post/s]

Error: 'ListingGenerator' object is not subscriptable. Waiting and retrying...


Processing posts: 1833post [00:30, 86.49post/s]                                 

Error: 'ListingGenerator' object is not subscriptable. Waiting and retrying...


Processing posts: 1864post [00:33, 54.96post/s]


KeyboardInterrupt: 

In [29]:
import praw
import pandas as pd
from datetime import datetime
from tqdm import tqdm

# Initialize PRAW Reddit instance
reddit = praw.Reddit(
    client_id=REDDIT_CLIENT_ID,
    client_secret=REDDIT_CLIENT_SECRET,
    user_agent=REDDIT_USER_AGENT
)

# Parameters
subreddits = ["breastcancer", "ovariancancer", "prostatecancer", "coloncancer", "lungcancer", "cancer"]  # Add more if you'd like
keywords = ["death", "dying", "palliative", "hospice", "terminal", "metastatic", "metastasized", "metastasis", "stage IV", "stage 4", "eol", "end of life", "comfort care"]
post_limit = 1000
posts = []

# Loop through each subreddit
for subreddit_name in subreddits:
    print(f"Scanning the {post_limit} most recent posts from r/{subreddit_name}...")

    try:
        submissions = reddit.subreddit(subreddit_name).new(limit=post_limit)

        with tqdm(total=post_limit, desc=f"r/{subreddit_name}", unit="post") as pbar:
            for submission in submissions:
                pbar.update(1)
                content = f"{submission.title.lower()} {submission.selftext.lower()}"
                if any(keyword in content for keyword in keywords):
                    posts.append({
                        "Subreddit": subreddit_name,
                        "Title": submission.title,
                        "Text": submission.selftext,
                        "Date": datetime.utcfromtimestamp(submission.created_utc).strftime('%Y-%m-%d %H:%M:%S'),
                        "URL": submission.url
                    })
            time.sleep(3)
    except Exception as e:
        print(f"Error while processing r/{subreddit_name}: {e}")

# Save all results to a single CSV
df = pd.DataFrame(posts)
df.to_csv("combined_death_related_posts.csv", index=False)
print(f"Saved {len(posts)} matching posts from all subreddits to combined_death_related_posts.csv")

Scanning the 1000 most recent posts from r/breastcancer...


r/breastcancer: 100%|█████████████████████▉| 999/1000 [00:16<00:00, 61.09post/s]


Scanning the 1000 most recent posts from r/ovariancancer...


r/ovariancancer:  99%|████████████████████▊| 994/1000 [00:16<00:00, 60.98post/s]


Scanning the 1000 most recent posts from r/prostatecancer...


...

Scanning the 1000 most recent posts from r/lungcancer...


r/lungcancer:  98%|███████████████████████▋| 985/1000 [00:15<00:00, 62.49post/s]


Scanning the 1000 most recent posts from r/cancer...


r/cancer:  93%|██████████████████████████  | 932/1000 [00:15<00:01, 61.39post/s]

Saved 1020 matching posts from all subreddits to combined_death_related_posts.csv


In [25]:
df

,Title,Text,Date,URL
0,Just Diagnosed,Just diagnosed with TNBC.\n\nI am 58 and 7 yea...,2025-04-03 01:24:12,https://www.reddit.com/r/breastcancer/comments...
1,Single liver metastasis?,"Hi all.\n\nNewly diagnosed with IDC, small pri...",2025-04-03 00:49:31,https://www.reddit.com/r/breastcancer/comments...
2,To Reconstruct or Not?,"33F with DCIS in my leftie, genetic mutation (...",2025-04-02 19:45:30,https://www.reddit.com/r/breastcancer/comments...
3,Do you feel alone with your diagnosis?,"Hi, I am 44, diagnosed with stage 3, ++-, duct...",2025-04-02 11:02:05,https://www.reddit.com/r/breastcancer/comments...
4,A Bottle of Pills,"Previous Posts: (12) [It's not if, it's when.....",2025-04-02 01:56:59,https://www.reddit.com/r/breastcancer/comments...
5,Choose your own f## up adventure,Here we go. TNBC. Another mindfuck. While on c...,2025-04-01 20:37:54,https://www.reddit.com/r/breastcancer/comments...
6,A Poem by Me,idk! i’ve been into writing throughout treatme...,2025-04-01 02:24:58,https://www.reddit.com/r/breastcancer/comments...
7,Rant: my town is poison.,"Don't move here, don't even visit. Don't even ...",2025-04-01 00:21:07,https://www.reddit.com/r/breastcancer/comments...
8,Dropping Verzenio?,"Hey fellow journey folk,\n\nTotal shite - righ...",2025-03-31 22:27:13,https://www.reddit.com/r/breastcancer/comments...
9,Sharing my story,"Hello everybody,\n\nI am 37, married, two chil...",2025-03-30 22:10:19,https://www.reddit.com/r/breastcancer/comments...
